In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd

from matplotlib import gridspec
from IPython.display import HTML

plt.xkcd()

HTML("""
<style>
.toggle_box {
    position: fixed;
    align: center;
    padding: 2px;
    top: 17%;
    left: 94%;
    opacity: 0.6;
    z-index: 10000;
}
.toggle_button {
    padding: 1px 3px 0 1px;
}
</style>

<script>
var code_show = true;

function code_toggle() {
  if (code_show) {
    $('div.input').hide('linear');
    $('img#code-toggle')
      .attr('src', 'assets/expand.png')
      .attr('title', 'click to display code cells');
  } else {
    $('div.input').show('linear');
    $('img#code-toggle')
      .attr('src', 'assets/collapse.png')
      .attr('title', 'click to hide code cells');
  }
  code_show = !code_show;
}
</script>

<div class='toggle_box'>
  <button class='toggle_button' onclick='javascript:code_toggle()'>
    <img id='code-toggle' src='assets/collapse.png' title='click to hide code cells'/>
  </button>
</div>""")

# Gradient Descent for Linear Regression

## Introduction

In machine learning, gradient descent is used to optimize a cost function, by measuring the difference between a prediction $\hat{y}$ given by a hypothesis function $h_{\theta}$ and the actual value $y$. This difference between $h_{\theta}$ and $y$ is modelled with cost function $J(\theta)$.

Optimizing $J(\theta)$ helps us find a set of model parameters $\theta$ that predict $y$ with the least amount of error.

In terms of the graph below, gradient descent tries to find values for parameter $\theta_{0}$ and $\theta_{1}$ that minimize the cost surface $J(\theta)$.

![gradient descent](assets/3Dsurface.png "What values for parameter θ₀ and θ₁ minimize the cost surface J(θ)?")

The notebook gives a very brief example of using gradient descent in linear regression.

This example will cover:

1. exploratory data analysis
1. deciding on a hypothesis function
1. deriving the corresponding cost function and gradient function
1. implement gradient descent
1. confirm and visualize the results

The code in our example uses:

* [NumPy](http://www.numpy.org/) - a package for scientific / mathematical computing
* [Pandas](http://pandas.pydata.org/) - a Python data analysis library

----

## Data Exploration

We begin by loading our data on file into a [`pandas.Dataframe`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html).

We can get a better idea about the data by using [`pandas.DataFrame.shape`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) and [`pandas.DataFrame.head`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html)

In [ ]:
data = pd.read_csv('data/ex1data1.txt',
                   header=None,
                   names=['x', 'y'])

print('{} rows, {} columns\n'.format(*data.shape))
print('Here are the first 10 rows:\n{}'.format(data.head(10)))

In [ ]:
print('{}'.format(data.describe()))

In [ ]:
data.y.size

Visualing the data as $x,y$ coordinates in a scatter plot will help us decide on a hypothesis function for our model.

In [ ]:
plt.figure(figsize=(18,6))
gs = gridspec.GridSpec(1, 2, width_ratios=[1,1])
graph0 = plt.subplot(gs[0])
graph0.scatter(data.x, data.y, color='#3182bd')
graph0.set_xlabel('x', fontsize=14)
graph0.set_ylabel('y', fontsize=14)
graph0.set_title(r'Scatter plot of the dataset', fontsize=16)
graph0.set_facecolor('0.88')

----

### Linear regression and hypothesis function $h_{\theta}$

We would like to derive a model where, given a new value $x$, we could confidently predict the corresponding $y$. Gaining a clue from the scatter plot above, perhaps a _line that best fits the points in our data set_ will serve as the hypothesis function for our model.

Recall that the slope-intercept formula for a straight line is

>\begin{align}
 y &= b + mx
>\end{align}

The model we are looking for can then be represented with the following hypothesis function:

>\begin{align}
  h_{\theta}(x) &= \theta_{0} + \theta_{1}x
>\end{align}

The model parameter in question, called collectively $\theta$, are:
* $\theta_{0}$, $y$-intercept
* $\theta_{1}$, slope $m$

But how do we go about finding appropriate values for $\theta$ that will provide us the best possible model?

### Computing cost function $J(\theta)$



In [ ]:
def cost(theta, X, y):
    theta = np.matrix(theta)
    
    m = X.shape[0]
    
    y_hat = np.dot(X, theta.T).sum(axis=1)
    
    cost = (1. / (2*m)) * np.square(y_hat - y).sum()
    
    return cost


In [ ]:
def gradient(theta, X, y):
    theta = np.matrix(theta)
    
    m = X.shape[0]
    
    y_hat = np.dot(X, theta.T).sum(axis=1)
    err = (y_hat - y).ravel()
    
    deriv_0 = (1./m) * np.multiply(err, X[:, 0]).sum()
    deriv_1 = (1./m) * np.multiply(err, X[:, 1]).sum()
    
    return deriv_0, deriv_1


In [ ]:
NUM_ITERS = 1000    # number of iterations for gradient descent
ALPHA = 1e-2        # step size scaling

def gradient_descent(theta, X, y):
    m = X.shape[0]
    
    cost_history = np.zeros(shape=(NUM_ITERS, 1))
    
    theta_history = np.zeros(shape=(NUM_ITERS, 2))
    
    for i in range(NUM_ITERS):
        deriv_0, deriv_1 = gradient(theta, X, y)
        
        theta[0] -= ALPHA * deriv_0
        theta[1] -= ALPHA * deriv_1
        
        theta_history[i,0] = theta[0]
        theta_history[i,1] = theta[1]
        
        cost_history[i,0] = cost(theta, X, y)
        
    return theta, cost_history, theta_history


In [ ]:
# prepare training data, add bias column
X = data.values[:,0:1]
X = np.insert(X, 0, np.ones(X.shape[1]), axis=1)

y = data.values[:,1:2]

theta = np.zeros(2)

# here are the first 5 rows of the training data
print(X[0:5,:])


In [ ]:
print('With the following initial θ values:')
print(r'θ₀ (y-intercept): {}'.format(theta[0]))
print(r'θ₁ (slope): {}'.format(theta[1]))
print('initial cost = {}'.format(cost(theta, X, y)))


In [ ]:
theta, cost_history, theta_history = gradient_descent(theta, X, y)

print('After {} iterations of gradient descent:'.format(NUM_ITERS))
print(r'θ₀ (y-intercept): {}'.format(theta[0]))
print(r'θ₁ (slope): {}'.format(theta[1]))
print('initial cost = {}'.format(cost(theta, X, y)))


In [ ]:
# predict y value for x = 3.5 (adding in bias node value +1)
predict1 = np.array([1, 3.5]).dot(theta)
print('For x = 3.5, our model predict y = {}'.format(predict1))


In [ ]:
# predict y value for x = 7.0 (adding in bias node value +1)
predict2 = np.array([1, 7.0]).dot(theta)
print('For x = 7.0, our model predict y = {}'.format(predict2))


In [ ]:
fx = theta_history[:,0]
fb = theta_history[:,1]

colseq = ['#fd8d3c', '#f16913', '#d94801', '#a63603', '#7f2704']

plt.figure(figsize=(18,6))
gs = gridspec.GridSpec(1, 2, width_ratios=[1,1])

# scatter plot
graph0 = plt.subplot(gs[0])
graph0.scatter(X[:,1:2], y, color='#3182bd')

for e in zip([0, 200, 400, 600, 800], [0.3, 0.4, 0.5, 0.6, 0.7]):
    i, a = e
    graph0.plot(data.values[:, 0:1],
                X.dot([fx[i], fb[i]]).flatten(),
                color=colseq[0],
                alpha=a,
                ls=':')

result = X.dot(theta).flatten()
graph0.plot(data.values[:,0:1], result, color='#7f2704')

graph0.set_xlabel(r'x', fontsize=14)
graph0.set_ylabel(r'y', fontsize=14)
graph0.set_title(r'Optimizing $\theta_0, \theta_1$ with gradient descent',
                 fontsize=16)
graph0.set_facecolor('0.88')


In [ ]:
plt.figure(figsize=(18,6))
gs = gridspec.GridSpec(1, 2, width_ratios=[1,1])

theta0_vals = np.linspace(-10, 10, 100)
theta1_vals = np.linspace(-1, 4, 100)

J_vals = np.zeros(shape=(theta0_vals.size, theta1_vals.size))

for t1, e1 in enumerate(theta0_vals):
    for t2, e2 in enumerate(theta1_vals):
        thetaT = np.zeros(2)
        thetaT[0] = e1
        thetaT[1] = e2
        J_vals[t1,t2] = cost(thetaT, X,  y)

J_vals = J_vals.T

graph1 = plt.subplot(gs[0])

levels = np.logspace(-2, 3, 20).size
colormap = plt.cm.get_cmap('coolwarm', levels-1)

plt.contour(theta0_vals,
            theta1_vals,
            J_vals,
            np.logspace(-2, 3, 20),
            cmap=colormap)

graph1.set_xlabel(r'$\theta_0$', fontsize=14)
graph1.set_ylabel(r'$\theta_1$', fontsize=14)
graph1.scatter(fx[0::200], fb[0::200], marker='x', c=colseq)
graph1.scatter(theta[0], theta[1], color='#7f2704')
graph1.set_title(r'Traversing contour with gradient descent on $J(\theta_0,\theta_1)$',
                 fontsize=16)
graph1.set_facecolor('0.88')


In [ ]:
from scipy.optimize import minimize

res = minimize(cost,
               np.zeros(2),
               args=(X,y),
               method='TNC',
               jac=gradient)

print('Using scipy.optimize.minimize(method="TNC") function:')
print(r'θ₀ (y-intercept): {}'.format(res.x[0]))
print(r'θ₁ (slope): {}'.format(res.x[1]))
print('cost = {}'.format(cost(res.x, X, y)))

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X[:, 1:2], y)

theta0 = lr.intercept_.flatten()[0]
theta1 = lr.coef_.flatten()[0]

print('Using sklearn.linear_mode.LinearRegression:')
print(r'θ₀ (y-intercept): {}'.format(theta0.item()))
print(r'θ₁ (slope): {}'.format(theta1))
print('cost = {}'.format(cost([theta0, theta1], X, y)))


In [ ]:
gramian = np.linalg.inv(np.dot(X.T, X))
moment = np.dot(X.T, y)

mpp_theta = np.dot(gramian, moment)

print('Using OLS (Moore-Penrose pseudoinverse):')
print(r'θ₀ (y-intercept): {}'.format(mpp_theta[0].item()))
print(r'θ₁ (slope): {}'.format(mpp_theta[1].item()))
print('cost = {}'.format(cost(mpp_theta.reshape((2)), X, y)))


----